## 셀레니움으로 스크래핑

In [1]:
import pandas as pd 
from selenium import webdriver
from bs4 import BeautifulSoup as bs 
import requests
from fake_useragent import UserAgent
pd.set_option('display.max_colwidth', None)

In [2]:
url = 'https://www.melon.com/chart/hot100/index.htm'
chrome_path = 'C:/Users/rnlay/.wdm/drivers/chromedriver/win64/118.0.5993.89/chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument(f'--executable-path={chrome_path}')

driver = webdriver.Chrome(options=options)

driver.implicitly_wait(10)
driver.get(url)

In [3]:
html = driver.page_source
soup = bs(html, 'html.parser')
song_soup = soup.select('tbody > tr')

In [4]:
song_lst = []

for song in song_soup:
    song_title = song.find('div', class_= 'ellipsis rank01').get_text()
    song_no = song.find('div', class_='ellipsis rank01').find('a')['href'].split("'")[2]
    song_artist = song.find('span', class_= 'checkEllipsis').get_text()
    song_album = song.find('div', class_= 'ellipsis rank03').get_text()
    song_likes = song.find('button', class_ = 'button_etc like').get_text()

    song_lst.append([song_no,
                     song_title.replace('\n', ''),
                     song_artist.replace('\n', ''),
                     song_album.replace('\n', ''),
                     song_likes.replace('\n', '').replace('좋아요총건수', '')])

In [5]:
driver.quit()

df = pd.DataFrame(song_lst, columns = ['곡ID', '곡명', '가수', '앨범', '좋아요'])

df['곡ID'] = df['곡ID'].str.extract(r'(\d+)')
df['순위'] = range(1, len(df) + 1)
df = df[['순위', '곡ID','곡명', '가수', '앨범', '좋아요']]
df.head()

,순위,곡ID,곡명,가수,앨범,좋아요
0,1,37138469,나는 아픈 건 딱 질색이니까,(여자)아이들,2,"89,083"
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,"98,658"
2,3,37194943,밤양갱,비비 (BIBI),밤양갱,"122,074"
3,4,37225604,EASY,LE SSERAFIM (르세라핌),EASY,"60,757"
4,5,37145732,Love wins all,아이유,The Winning,"150,367"


## 가사 추출

In [6]:
def get_lyrics(song_id):
    headers = {"user-agent": UserAgent().chrome}
    url = f"https://www.melon.com/song/detail.htm?songId={song_id}"
    res = requests.get(url, headers=headers)
    
    if res.status_code == 200:
        soup = bs(res.text, 'html.parser')
        lyrics_tag = soup.find('div', class_='lyric')
        
        if lyrics_tag:
            lyrics = lyrics_tag.get_text(separator=' ', strip=True)
            return lyrics

    return None

df2 = pd.DataFrame(df)
df2['가사'] = df2['곡ID'].apply(get_lyrics)
df2.head(2)

,순위,곡ID,곡명,가수,앨범,좋아요,가사
0,1,37138469,나는 아픈 건 딱 질색이니까,(여자)아이들,2,"89,083",오늘도 아침엔 입에 빵을 물고 똑같이 하루를 시작하고 온종일 한 손엔 아이스 아메리카노 피곤해 죽겠네 지하철 속 이 장면 어제 꿈에서 봤나 아참 매일이지 지나치고 바쁜 이 삶에 그냥 흔한 날에 그 애를 보고 말야 평온했던 하늘이 무너지고 어둡던 눈앞이 붉어지며 뭔가 잊고 온 게 있는 것 같아 괜히 이상하게 막 울 것만 같고 그냥 지나치는 게 나을 것 같아 나는 생각은 딱 질색이니까 카페인으로 잡은 정신은 빠졌고 하루 종일 신경 쓰여 토할 것 같아 저녁이 돼도 배고픔까지 까먹고 그치 이상하지 근데 말야 있잖아 처음 본 순간 뭐라 할까 그립달까 나도 웃긴데 말야 평온했던 하늘이 무너지고 어둡던 눈앞이 붉어지며 뭔가 잊고 온 게 있는 것 같아 괜히 이상하게 막 울 것만 같고 그냥 지나치는 게 나을 것 같아 나는 생각은 딱 질색이니까 오랫동안 나를 아는 슬픈 표정을 하고 Oh 흔적 없는 기억 밖 혹 과거에 미래에 딴 차원에 세계에 1 2 3 4 5 6 7 8 평온했던 하늘이 무너지고 어둡던 눈앞이 붉어져도 다시 놓쳐버리는 것만 같아 괜히 이상하게 막 울 것만 같고 그냥 지나치는 게 나을 것 같아 나는 생각은 딱 질색이니까 아냐 지나치는 게 나을 것 같아 나는 아픈 건 딱 질색이니까
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,"98,658","Ay ay ay ay ay 거울 속에 내 표정 봐 봐 느낌 So good 기다려온 D-day 연습했던 손든 인사도 그대로 하면 돼 Hairstyle check하고 한 번 Turn around 발걸음은 매일 걷던 그 길로 계획은 완벽 빨리 말 걸어보고 싶어, Hey Woo 문 앞에서 셋을 세어본다, Yeh (셋, 둘, 하나) 첫 만남은 너무 어려워 계획대로 되는 게 없어서 첫 만남은 너무 어려워 내 이름은 말야 Hey, 안녕, 첫 마디를 건넬 때 주변 소린 Canceled 네 말소리는 Playlist Yeh, 질문은 나의 용기, 알려줘 너의 “이름이 뭐야?” 너와 내 거리는 세 걸음 남았어, Yeh (셋, 둘, 하나) 첫 만남은 너무 어려워 계획대로 되는 게 없어서 첫 만남은 너무 어려워 내 이름은 말야 이 순간, Feels so wonderful 조금은 뚝딱거려도 어색한 인사까지도 너와 나의 첫 만남 우리의 사이 Beautiful 내일도 내일모레도 기억해, 영원히 반짝일 순간 Wait wait! Na na na- 이렇게 만나서 반가워 내일 또 봐 안녕"


In [7]:
df2.to_csv("가사.csv", encoding="utf-8-sig")